In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
def get_train_data():
    data_train = pd.read_csv('../Datasets/Fashion MNIST/fashion-mnist_train.csv')
    X_train = data_train.drop(['label'], axis=1).to_numpy()
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))  # Обновленный формат

    y_train = data_train['label'].to_numpy()
    # print('x_train\n', X_train[:5])
    # print()
    # print('y_train\n', y_train[:5])
    return X_train, y_train

In [3]:
X_train, y_train = get_train_data()

In [4]:
X_train

array([[[[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        ...,

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]]],


       [[[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         ...,
         [  0],
         [

In [5]:
y_train

array([2, 9, 6, ..., 8, 8, 7], dtype=int64)

In [32]:
class CNN:
    def __init__(self, input_size, num_classes, count_convolutional_layer):
        self.input_size = input_size
        self.num_classes = num_classes
        self.count_convolutional_layer = count_convolutional_layer
        self.weight_count_convolutional = self.get_weight_count_convolutional()
    
    def get_weight_count_convolutional(self):
        weight_count_convolutional = [[random.uniform(0.3, 0.6), random.uniform(0.3, 0.6), random.uniform(0.3, 0.6)] for i in range(3)]
        print(weight_count_convolutional)
        return weight_count_convolutional

    def convolve2d(self, input_data):
        height, width, _ = input_data.shape
        filter_height, filter_width = len(self.weight_count_convolutional), len(self.weight_count_convolutional[0])
        print('input_data', input_data.shape)
        print('weight_count_convolutional', filter_height, filter_width)
        print(X[10000000])
        for i in range(output_height):
            for j in range(output_width):
                for k in range(num_filters):
                    output_data[i, j, k] = np.sum(
        #                 input_data[i:i + filter_height, j:j + filter_width, :] * filters[:, :, :, k])
        # return output_data

    def train(self, X, y, epoch):
        for eph in range(epoch):
            for index_training_example, training_example in enumerate(X):
                self.convolve2d(training_example)
                
        


In [33]:
simple_conv_net = CNN(
    input_size=X_train[0].shape[0], 
    num_classes=len(np.unique(y_train)),
    count_convolutional_layer = 1)
simple_conv_net.train(X=X_train, y=y_train, epoch=1)

[[0.5332837399933951, 0.5960225670257442, 0.5856895934741795], [0.49741662708516843, 0.5910869415643729, 0.4886715843627462], [0.5498700502845246, 0.32705063790920325, 0.30430311088491824]]
input_data (28, 28, 1)
weight_count_convolutional 3 3


NameError: name 'X' is not defined